# <center>Trabajo Práctico - Twitter API<center>
***


## Introducción
En éste trabajo presentamos una pequeña aplicación encargada de la recolección y presentación de información proporcionada por la **API** de **Twitter**. Ésta misma buscará aquellos tweets que nos resulten interesantes según la temáctica que hayamos elegido.


***

## Query
A la hora de elegir un contenido para la investigación nos tomamos el trabajo de seleccionar un temática concurrente para el desarrollo de una consulta firme y robusta.
Decidimos utilisar el tópico de cambio climático, y sus temas relacionados, para la construcción de la siguiente QUERY.

***QUERY = "cambio climático OR sequías OR calentamiento global OR economía circular OR espacios verde OR protección ambiental lang:es -is:retweet"***

Con ésta misma anduvimos recolecantdo una gran cantidad de tweets para el desarrollo de una estrucutura compuesta para el procesamiento, lectura y servicio de los datos almacenados.

***

## Recolección de Tweets

Con el siguiente código pudimos recolectar una cierta cantidad de Tweets. Modificamos el mismo, proporcionado por las presentaciones dadas en clase, para que sea más optimo para nuestra tarea. Incluimos una limpieza de las ***STREAM RULES***, esto mismo lo vimos importante debido a que, cada vez que realizabamos una nueva busqueda, estas mismas quedaban con la información de la busqueda previa. Como consecuencia, nos quedaba "información basura" para futuras consultas. También incluimos la hora en la que se inicia la búsqueda, la cantidad de Tweets recopilados hasta el momento, el tamaño del archivo, como también la datos de la duracióon de la prueba, entre otros datos que nos parecieron importantes.

In [ ]:
import os
import sys
import time
from datetime import datetime
from TwitterAPI import (
    TwitterAPI,
    TwitterOAuth,
    TwitterRequestError,
    TwitterConnectionError,
    HydrateType,
    OAuthType,
)
import json

def stream_tweets(query, expansions, tweet_fields, user_fields):

    datetimestart = datetime
    try:
        o = TwitterOAuth.read_file("credentials.txt")
        api = TwitterAPI(
            o.consumer_key,
            o.consumer_secret,
            auth_type=OAuthType.OAUTH2,
            api_version="2",
        )

        # DELETE STREAM RULES
        r = api.request("tweets/search/stream/rules", method_override="GET")
        rules = r.json()
        if "data" in rules:
            ids = list(map(lambda rule: rule["id"], rules["data"]))
            api.request("tweets/search/stream/rules", {"delete": {"ids": ids}})

        # ADD STREAM RULES
        r = api.request("tweets/search/stream/rules", {"add": [{"value": query}]})
        print(f"[{r.status_code}] RULE ADDED: {json.dumps(r.json(), indent=2)}\n")
        if r.status_code != 201:
            exit()

        # GET STREAM RULES

        r = api.request("tweets/search/stream/rules", method_override="GET")
        print(f"[{r.status_code}] RULES: {json.dumps(r.json(), indent=2)}\n")
        if r.status_code != 200:
            exit()

        # START STREAM

        r = api.request(
            "tweets/search/stream",
            {
                "expansions": expansions,
                "tweet.fields": tweet_fields,
                "user.fields": user_fields,
            },
            hydrate_type=HydrateType.APPEND,
        )

        if r.status_code != 200:
            exit()
        
        if not os.path.exists("data.json"):
            open("data.json", "x", encoding="utf-8")

        with open("data.json", "r+", encoding="utf-8") as file:
            cantidad_tweets = len(file.readlines())
            file.seek(0, os.SEEK_END)

            print("------------------------------------------------------------")
            datetimestart = datetime.now()
            print(
                (
                    "Proceso de recopilación iniciado: "
                    + datetimestart.strftime("%d/%m/%Y %H:%M:%S")
                )
            )

            for item in r:
                json.dump(item, file, ensure_ascii=False, indent=None)
                file.write("\n")

                cantidad_tweets += 1
                sys.stdout.write(
                    f"\rTamaño actual del archivo: {file.tell() / 1000} kb | Cantidad de tweets: {cantidad_tweets}"
                )

    except KeyboardInterrupt:
        datetimeend = datetime.now()
        print("\nProceso terminado: " + datetimeend.strftime("%d/%m/%Y %H:%M:%S"))
        datetimeend = datetimeend - datetimestart
        print("Duración de la prueba " + (str(datetimeend)) + " horas/minutos/segundos")
        print("------------------------------------------------------------")

    except TwitterRequestError as e:
        print(f"\n{e.status_code}")
        for msg in iter(e):
            print(msg)
    except TwitterConnectionError as e:
        print(e)
        print("tce")
    except Exception as e:
        print(e)
        print("e")


QUERY = "cambio climático OR sequías OR calentamiento global OR economía circular OR espacios verde OR protección ambiental lang:es -is:retweet"
EXPANSIONS = "author_id,referenced_tweets.id,referenced_tweets.id.author_id,in_reply_to_user_id,attachments.media_keys,attachments.poll_ids,geo.place_id,entities.mentions.username"
TWEET_FIELDS = "author_id,conversation_id,created_at,entities,geo,id,lang,public_metrics,source,text"
USER_FIELDS = "created_at,description,entities,location,name,profile_image_url,public_metrics,url,username"

stream_tweets(QUERY, EXPANSIONS, TWEET_FIELDS, USER_FIELDS)

***

## Consulta por fechas y horas

En ésta sección decidimos utilizar la estructura de ***B-Tree***

![alt text](b-tree.webp "B-Tree")